# Proyek Analisis Sentimen Aplikasi BRIMO

In [ ]:
import pandas as pd

app_reviews_df = pd.read_csv('hasil_scraping.csv')
 
app_reviews_df.shape

(72000, 11)

In [ ]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,5231b04b-3dbc-4b7f-862b-065af99bbae4,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"aplikasi buruk, merugikan, abal abal, duit say...",1,67,2.92.1,2026-01-21 11:12:22,"Hai Sobat BRI, mohon maaf atas kendala top up ...",2026-01-21 11:06:59,2.92.1
1,8fc2ddb6-2946-40f8-a0e7-9d4ad9ac0c93,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bank besar,tp aplikasinya payah.KTP baret diki...",1,51,2.92.1,2026-01-20 15:43:11,"Hai Sobat BRI, mohon maaf atas kendala yang di...",2026-01-20 16:03:11,2.92.1
2,c6fd8082-7ead-4e3f-b14d-4685733ee76f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"setiap login disuruh masukan KTP lagi, jadi se...",4,63,2.92.0,2026-01-17 05:32:37,"Hai Sobat BRI, mohon maaf atas kendala yang di...",2026-01-17 06:36:49,2.92.0
3,1970f46c-5013-425f-949d-37565fb7343e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Kenapa akhir2 ini brimo di hp saya, selalu set...",4,270,2.92.0,2026-01-09 12:59:50,"Hai Sobat BRI, mohon maaf atas ketidaknyamanan...",2026-01-09 13:19:42,2.92.0
4,42de095a-495f-4bd7-bfed-8ed54403c5c0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya kurangi 1 bintang sebab untuk login finge...,4,14,2.92.1,2026-01-19 19:43:15,"Hai Sobat BRI, terima kasih atas ulasannya. Se...",2024-05-04 16:35:01,2.92.1


In [ ]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72000 entries, 0 to 71999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              72000 non-null  object
 1   userName              72000 non-null  object
 2   userImage             72000 non-null  object
 3   content               72000 non-null  object
 4   score                 72000 non-null  int64 
 5   thumbsUpCount         72000 non-null  int64 
 6   reviewCreatedVersion  60689 non-null  object
 7   at                    72000 non-null  object
 8   replyContent          71875 non-null  object
 9   repliedAt             71875 non-null  object
 10  appVersion            60689 non-null  object
dtypes: int64(2), object(9)
memory usage: 6.0+ MB


### cleaning data

In [ ]:
# Pilih kolom yang relevan untuk analisis sentimen
columns_to_keep = ['reviewId', 'userName', 'content', 'score', 'thumbsUpCount', 
                   'reviewCreatedVersion', 'at', 'app_name', 'app_id']

# Filter kolom yang ada
available_columns = [col for col in columns_to_keep if col in app_reviews_df.columns]
df = app_reviews_df[available_columns].copy()

# Rename kolom agar lebih mudah dibaca
df = df.rename(columns={
    'reviewId': 'review_id',
    'userName': 'user_name', 
    'content': 'review_text',
    'score': 'rating',
    'thumbsUpCount': 'thumbs_up',
    'reviewCreatedVersion': 'app_version',
    'at': 'review_date'
})

df.shape

(72000, 7)

In [ ]:
# Cek missing values
df.isnull().sum()

review_id          0
user_name          0
review_text        0
rating             0
thumbs_up          0
app_version    11311
review_date        0
dtype: int64

In [ ]:
df = df[df['review_text'].str.strip() != '']

In [ ]:
duplicates = df.duplicated(subset=['review_text']).sum()
duplicates

np.int64(261)

In [ ]:
# Hapus duplikat berdasarkan kolom 'review_text'
df = df.drop_duplicates(subset=['review_text'])

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
print(f"TOTAL DATA SETELAH CLEANING: {len(df):,} reviews")

# Cek apakah memenuhi syarat minimal 10000
if len(df) >= 10000:
    print("Memenuhi syarat minimal 10.000 sampel!")
else:
    print("Data kurang dari 10.000. Perlu scraping tambahan.")

TOTAL DATA SETELAH CLEANING: 71,739 reviews
Memenuhi syarat minimal 10.000 sampel!


### pre processing text
#### pelabelan data dengan lexicon dan ekstraksi fitur

In [ ]:
POSITIVE_LEXICON = {
    'bagus', 'baik', 'hebat', 'luar biasa', 'sempurna', 'mantap', 'mantab', 'mantul',
    'keren', 'canggih', 'istimewa', 'unggul', 'prima', 'optimal', 'maksimal',
    'fantastis', 'menakjubkan', 'mengagumkan', 'memukau', 'spektakuler',
    'brilian', 'cemerlang', 'gemilang', 'mumpuni', 'andal', 'handal',

    'puas', 'senang', 'suka', 'cinta', 'sayang', 'kagum', 'takjub', 'terpesona',
    'bahagia', 'gembira', 'riang', 'sukacita', 'antusias', 'semangat',
    'nyaman', 'tenang', 'aman', 'tenteram', 'damai', 'lega', 'plong',

    'cepat', 'cepet', 'kilat', 'gesit', 'sigap', 'tanggap', 'responsif',
    'mudah', 'gampang', 'simpel', 'sederhana', 'praktis', 'efisien', 'efektif',
    'lancar', 'mulus', 'smooth', 'stabil', 'konsisten', 'reliable',
    'lengkap', 'komplit', 'komprehensif', 'detail', 'rinci',
    'murah', 'terjangkau', 'ekonomis', 'hemat', 'worth', 'worthit',
    'gratis', 'free', 'bonus', 'promo', 'diskon', 'cashback',

    'terima kasih', 'terimakasih', 'makasih', 'thanks', 'thx', 'tq',
    'apresiasi', 'hargai', 'salut', 'acungi jempol', 'top', 'jos',
    'recommended', 'recommend', 'rekomendasi', 'rekomen',
    'terbaik', 'best', 'nomor satu', 'juara', 'champion',

    'membantu', 'menolong', 'mendukung', 'memudahkan', 'mempermudah',
    'memuaskan', 'menyenangkan', 'menghibur', 'menarik',
    'berfungsi', 'bekerja', 'jalan', 'berjalan', 'aktif', 'hidup',
    'berhasil', 'sukses', 'tercapai', 'terwujud', 'terlaksana',
    'meningkat', 'berkembang', 'maju', 'progress', 'improve',

    'mantep', 'mantabs', 'josss', 'joss', 'topcer', 'topp', 'oks', 'okee',
    'asik', 'asyik', 'sip', 'siip', 'oke banget', 'kece', 'keceh',
    'niceee', 'nice', 'good', 'great', 'awesome', 'amazing', 'excellent',
    'perfect', 'wonderful', 'beautiful', 'love', 'lovely',

    'update bagus', 'fitur lengkap', 'tampilan menarik', 'ui bagus', 'ux bagus',
    'loading cepat', 'ringan', 'tidak lemot', 'tidak lag', 'anti lag',
    'driver ramah', 'pelayanan bagus', 'respon cepat', 'fast response',
    'pengiriman cepat', 'tepat waktu', 'on time', 'sesuai', 'akurat',
    'aman', 'terpercaya', 'trusted', 'terjamin', 'garansi',
}

NEGATIVE_LEXICON = {
    'jelek', 'buruk', 'busuk', 'bobrok', 'rusak', 'hancur', 'ancur',
    'parah', 'payah', 'sampah', 'garbage', 'trash', 'zonk', 'zonkk',
    'mengecewakan', 'kecewa', 'menyesal', 'nyesel', 'rugi', 'sia-sia',

    'tidak puas', 'tidak suka', 'benci', 'muak', 'jijik', 'kesal',
    'marah', 'emosi', 'geram', 'murka', 'berang', 'jengkel',
    'frustrasi', 'frustasi', 'stress', 'stres', 'pusing', 'bingung',
    'sedih', 'kecewa berat', 'menyedihkan', 'miris', 'tragis',

    'error', 'eror', 'bug', 'bugs', 'crash', 'hang', 'freeze',
    'lemot', 'lambat', 'lelet', 'lama', 'loading lama', 'lag', 'ngelag',
    'force close', 'fc', 'keluar sendiri', 'menutup sendiri', 'mati',
    'tidak bisa', 'gabisa', 'gagal', 'failed', 'fail',
    'tidak jalan', 'tidak berfungsi', 'tidak bekerja', 'macet', 'stuck',
    'blank', 'kosong', 'hilang', 'lost', 'missing',

    'sulit', 'susah', 'ribet', 'rumit', 'kompleks', 'complicated',
    'membingungkan', 'confusing', 'tidak jelas', 'ambigu',
    'mahal', 'kemahalan', 'overpriced', 'tidak worth', 'boros',
    'tidak lengkap', 'minim', 'terbatas', 'limited',

    'tidak responsif', 'slow respon', 'slow response', 'lama respon',
    'tidak membantu', 'tidak berguna', 'useless', 'percuma',

    'bohong', 'tipu', 'nipu', 'penipuan', 'scam', 'fraud',
    'curang', 'tidak jujur', 'tidak adil', 'merugikan',

    'menghapus', 'hapus', 'uninstall', 'delete', 'buang',
    'menjengkelkan', 'menyebalkan', 'mengganggu',
    'merusak', 'menghancurkan', 'memperburuk', 'memperparah',

    'bangsat', 'kampret', 'sialan', 'brengsek', 'tolol', 'bodoh', 'goblok',
    'gak guna', 'gak berguna', 'sampah banget', 'parah banget',
    'kapok', 'ogah', 'males', 'malas', 'enggan',
    'worst', 'bad', 'terrible', 'horrible', 'awful', 'sucks', 'suck',

    'update jelek', 'update parah', 'makin jelek', 'makin buruk',
    'driver kasar', 'driver tidak ramah', 'pelayanan buruk',
    'pengiriman lama', 'terlambat', 'telat', 'tidak sesuai',
    'barang rusak', 'barang salah', 'tidak sampai', 'refund lama',
    'akun diblokir', 'banned', 'suspend', 'tidak bisa login',
}

NEGATION_WORDS = {
    'tidak', 'tak', 'bukan', 'belum', 'jangan', 'tanpa', 'tiada',
    'gak', 'ga', 'gk', 'ngga', 'nggak', 'kagak', 'kaga', 'enggak',
    'tdk', 'blm', 'jgn', 'gx', 'g',
    'never', 'no', 'not', 'none', 'nothing', 'neither', 'nobody'
}

SOFT_NEGATORS = {'kurang'}

INTENSIFIER_WORDS = {
    'sangat': 1.5, 'amat': 1.5, 'sekali': 1.5, 'banget': 1.5, 'bgt': 1.5,
    'super': 1.8, 'ultra': 1.8, 'extra': 1.5, 'ekstra': 1.5,
    'terlalu': 1.3, 'paling': 1.5, 'ter': 1.3,
    'luar biasa': 2.0, 'extremely': 2.0, 'very': 1.5, 'really': 1.5,
    'totally': 1.8, 'absolutely': 2.0, 'completely': 1.8,
    'sedikit': 0.5, 'agak': 0.7, 'cukup': 0.8, 'lumayan': 0.7,
    'hampir': 0.6
}

In [ ]:
STOPWORDS_ID = {
    'saya', 'aku', 'kamu', 'anda', 'dia', 'ia', 'kami', 'kita', 'mereka',
    'ini', 'itu', 'sini', 'situ', 'sana',
    'dan', 'atau', 'tetapi', 'tapi', 'namun', 'melainkan', 'padahal',
    'sedangkan', 'sementara', 'sebaliknya', 'bahkan', 'lagipula',    
    'di', 'ke', 'dari', 'pada', 'dalam', 'dengan', 'untuk', 'oleh',
    'tentang', 'terhadap', 'mengenai', 'sebagai', 'seperti', 'bagai',
    'sudah', 'telah', 'belum', 'akan', 'sedang', 'masih', 'pernah',
    'selalu', 'sering', 'jarang', 'kadang', 'hanya', 'cuma', 'saja',
    'juga', 'pun', 'lagi', 'lalu', 'kemudian', 'setelah', 'sebelum',
    'sambil', 'seraya', 'ketika', 'saat', 'waktu', 'bila', 'jika',
    'kalau', 'apabila', 'karena', 'sebab', 'akibat', 'sehingga',
    'supaya', 'agar', 'biar', 'meski', 'meskipun', 'walaupun', 'biarpun',
    'apa', 'siapa', 'mana', 'kapan', 'kenapa', 'mengapa', 'bagaimana', 'berapa',
    'ada', 'adalah', 'ialah', 'yaitu', 'yakni', 'merupakan',
    'bisa', 'dapat', 'mampu', 'sanggup', 'boleh', 'harus', 'wajib', 'perlu', 'mesti',
    'ingin', 'mau', 'hendak',
    'satu', 'dua', 'tiga', 'empat', 'lima', 'enam', 'tujuh', 'delapan', 'sembilan', 'sepuluh',
    'pertama', 'kedua', 'ketiga', 'banyak', 'sedikit', 'beberapa', 'semua', 'seluruh', 'setiap',
    'yg', 'nya', 'gak', 'ga', 'gk', 'ngga', 'nggak', 'kagak', 'enggak',
    'udah', 'udh', 'dah', 'sdh', 'uda',
    'aja', 'aj', 'doang', 'dong', 'deh', 'sih', 'nih', 'tuh', 'kok', 'kan', 'loh', 'lah',
    'banget', 'bgt', 'bngt', 'sangat', 'sekali', 'amat',
    'gitu', 'gini', 'begitu', 'begini',
    'gimana', 'gmn', 'gmna', 'bagaimana',
    'kayak', 'kyk', 'seperti',
    'emang', 'emg', 'memang',
    'aja', 'saja', 'cuma', 'cuman', 'hanya',
    'lg', 'lagi', 'lgi',
    'sm', 'sama', 'dgn', 'dengan',
    'tp', 'tpi', 'tapi', 'tetapi',
    'kalo', 'kl', 'klu', 'kalau',
    'krn', 'karena', 'soalnya',
    'utk', 'untuk', 'buat', 'uat',
    'jd', 'jadi', 'jadinya',
    'jg', 'jga', 'juga',
    'bs', 'bsa', 'bisa',
    'org', 'orang',
    'sy', 'sya', 'saya',
    'gw', 'gue', 'gua', 'w',
    'lu', 'lo', 'elu', 'elo',
    'app', 'apps', 'aplikasi', 'apk', 'application',
    'download', 'instal', 'install', 'update', 'version', 'versi',
    'bintang', 'star', 'rating', 'review', 'ulasan',
    'ya', 'yaa', 'yaaa', 'iya', 'iyaa', 'ok', 'oke', 'okay', 'okey',
    'oh', 'ohh', 'ah', 'ahh', 'eh', 'ehh', 'uh', 'uhh',
    'wah', 'wahh', 'wow', 'woww', 'yay', 'hore',
    'hmm', 'hmmm', 'mmm', 'emm',
    'haha', 'hahaha', 'hihi', 'hehe', 'wkwk', 'wkwkwk', 'kwkw', 'awkwk',
    'the', 'and', 'is', 'it', 'to', 'of', 'in', 'for', 'on', 'this', 'that',
    'be', 'are', 'was', 'were', 'been', 'have', 'has', 'had',
    'but', 'not', 'you', 'your', 'we', 'our', 'they', 'their',
    'so', 'if', 'or', 'as', 'at', 'by', 'an', 'no', 'yes',
    'very', 'just', 'only', 'also', 'even', 'still', 'too', 'more',
    'good', 'bad', 'nice', 'great', 'best', 'worst', 'please', 'thanks', 'thank'
}

SLANG_DICT = {
    'gk': 'tidak', 'gak': 'tidak', 'ga': 'tidak', 'tdk': 'tidak', 'gx': 'tidak',
    'ngga': 'tidak', 'nggak': 'tidak', 'kagak': 'tidak', 'kaga': 'tidak',
    'g': 'tidak', 'enggak': 'tidak', 'engga': 'tidak', 'tak': 'tidak',
    'udh': 'sudah', 'udah': 'sudah', 'sdh': 'sudah', 'uda': 'sudah', 'dah': 'sudah',
    'mantap': 'bagus', 'mantab': 'bagus', 'mantul': 'bagus', 'mantaps': 'bagus',
    'keren': 'bagus', 'cakep': 'bagus', 'oks': 'bagus', 'okee': 'bagus',
    'jos': 'bagus', 'josss': 'bagus', 'top': 'bagus', 'toppp': 'bagus',
    'recommended': 'rekomendasi', 'recommend': 'rekomendasi', 'recomended': 'rekomendasi',
    'worthed': 'worth', 'worthit': 'worth',
    'helpful': 'membantu', 'useful': 'berguna',
    'lancar': 'lancar', 'lancarr': 'lancar', 'lncar': 'lancar',
    'cepet': 'cepat', 'cpet': 'cepat', 'cpt': 'cepat',
    'gampang': 'mudah', 'gmpang': 'mudah', 'simpel': 'mudah', 'simple': 'mudah',
    'makasih': 'terima kasih', 'makasi': 'terima kasih', 'mksh': 'terima kasih',
    'thx': 'terima kasih', 'thanks': 'terima kasih', 'tq': 'terima kasih',
    'jelek': 'buruk', 'jlek': 'buruk', 'jelekk': 'buruk',
    'parah': 'buruk', 'parahhh': 'buruk', 'prah': 'buruk',
    'ancur': 'rusak', 'hancur': 'rusak', 'ancuurr': 'rusak',
    'payah': 'buruk', 'payahh': 'buruk',
    'zonk': 'buruk', 'zonkk': 'buruk',
    'sampah': 'buruk', 'smph': 'buruk',
    'lemot': 'lambat', 'lelet': 'lambat', 'lmot': 'lambat', 'lemott': 'lambat',
    'lama': 'lambat', 'lamaa': 'lambat', 'lma': 'lambat',
    'ribet': 'sulit', 'rbet': 'sulit', 'ribett': 'sulit',
    'susah': 'sulit', 'ssah': 'sulit', 'susahh': 'sulit',
    'error': 'error', 'eror': 'error', 'erorr': 'error',
    'bug': 'error', 'bugs': 'error', 'bugg': 'error',
    'crash': 'error', 'cras': 'error', 'crashh': 'error',
    'hang': 'error', 'ngehang': 'error', 'hangg': 'error',
    'force close': 'error', 'fc': 'error',
    'kecewa': 'kecewa', 'kcwa': 'kecewa', 'kcewaaa': 'kecewa',
    'nyesel': 'menyesal', 'nysel': 'menyesal', 'nyesell': 'menyesal',
    'rugi': 'rugi', 'rugii': 'rugi',
    'kapok': 'kapok', 'kapokk': 'kapok',
    'bgt': 'sangat', 'bngt': 'sangat', 'bngtt': 'sangat', 'bgtt': 'sangat',
    'bkn': 'bukan', 'blm': 'belum', 'blum': 'belum', 'blom': 'belum',
    'bs': 'bisa', 'bsa': 'bisa',
    'org': 'orang', 'orng': 'orang', 'orngg': 'orang',
    'bgus': 'bagus', 'bgs': 'bagus', 'baguss': 'bagus',
    'krg': 'kurang', 'kurang': 'kurang', 'krangg': 'kurang',
    'pdhl': 'padahal', 'pdhal': 'padahal',
    'smoga': 'semoga', 'moga': 'semoga', 'smga': 'semoga',
    'knp': 'kenapa', 'knapa': 'kenapa',
    'gmna': 'bagaimana', 'gmn': 'bagaimana',
    'bner': 'benar', 'bnr': 'benar', 'bener': 'benar',
    'slalu': 'selalu', 'sllu': 'selalu', 'sellalu': 'selalu',
    'pke': 'pakai', 'pkai': 'pakai', 'pk': 'pakai', 'make': 'pakai',
    'dri': 'dari', 'dr': 'dari',
    'yg': 'yang', 'yng': 'yang',
    'dgn': 'dengan', 'dngn': 'dengan',
    'utk': 'untuk', 'untk': 'untuk', 'u': 'untuk',
    'krn': 'karena', 'karna': 'karena', 'krna': 'karena',
    'klo': 'kalau', 'klu': 'kalau', 'kl': 'kalau',
    'jgn': 'jangan', 'jngn': 'jangan',
    'hrs': 'harus', 'hrus': 'harus',
    'trs': 'terus', 'trus': 'terus', 'teros': 'terus',
    'abis': 'habis', 'abiss': 'habis', 'abs': 'habis',
    'bru': 'baru', 'baru': 'baru',
    'ampe': 'sampai', 'sampe': 'sampai', 'smpe': 'sampai',
    'bgt': 'sangat', 'sgt': 'sangat',
    'bnyk': 'banyak', 'byk': 'banyak', 'bnyak': 'banyak'
}

In [ ]:
import re

_re_repeat = re.compile(r'(.)\1{2,}')

def _normalize_token(tok: str) -> str:
    tok = tok.lower().strip()
    tok = _re_repeat.sub(r'\1', tok)
    return tok

def _tokenize(text: str):
    if not text or not isinstance(text, str):
        return []
    raw = re.findall(r"[\w']+", text.lower())
    return [_normalize_token(t) for t in raw if t.strip()]


def calculate_sentiment_score(
    text,
    positive_lex,
    negative_lex,
    negation_words,
    intensifiers,
    neg_window=3,
    intens_window=2,
    soft_negators=None,
    soft_factor=0.6
):
    if not text or not isinstance(text, str):
        return {'positive': 0.0, 'negative': 0.0, 'score': 0.0, 'hits': 0}

    soft_negators = set(soft_negators or [])
    tokens = _tokenize(text)

    pos = 0.0
    neg = 0.0
    hits = 0

    neg_countdown = 0
    neg_scale = 1.0

    i = 0
    while i < len(tokens):
        tok = tokens[i]

        # handle negator/soft-negator + cek bigram negator kata
        if (tok in negation_words) or (tok in soft_negators):
            if i < len(tokens) - 1:
                bg = f"{tok} {tokens[i+1]}"
                if bg in positive_lex:
                    hits += 1
                    pos += 1.0
                    i += 2
                    neg_countdown = 0
                    neg_scale = 1.0
                    continue
                if bg in negative_lex:
                    hits += 1
                    neg += 1.0
                    i += 2
                    neg_countdown = 0
                    neg_scale = 1.0
                    continue

            neg_countdown = neg_window
            neg_scale = soft_factor if tok in soft_negators else 1.0
            i += 1
            continue

        # handle status negasi
        is_negated = neg_countdown > 0
        flip_scale = neg_scale if is_negated else 1.0

        if neg_countdown > 0:
            neg_countdown -= 1
            if neg_countdown == 0:
                neg_scale = 1.0

        # handle intensifier multiplier (unigram + bigram intensifier)
        intensity = 1.0
        start_j = max(0, i - intens_window)
        for j in range(start_j, i):
            tprev = tokens[j]
            if tprev in intensifiers:
                intensity *= float(intensifiers[tprev])

            if j - 1 >= 0:
                prev_bg = f"{tokens[j-1]} {tokens[j]}"
                if prev_bg in intensifiers:
                    intensity *= float(intensifiers[prev_bg])

        # handle bigram lexicon
        used = False
        if i < len(tokens) - 1:
            bg = f"{tokens[i]} {tokens[i+1]}"

            if bg in positive_lex:
                hits += 1
                if is_negated:
                    neg += 1.0 * intensity * flip_scale
                else:
                    pos += 1.0 * intensity
                i += 2
                used = True

            elif bg in negative_lex:
                hits += 1
                if is_negated:
                    pos += 0.8 * intensity * flip_scale
                else:
                    neg += 1.0 * intensity
                i += 2
                used = True

        if used:
            continue

        # handle nigram
        if tok in positive_lex:
            hits += 1
            if is_negated:
                neg += 1.0 * intensity * flip_scale
            else:
                pos += 1.0 * intensity

        elif tok in negative_lex:
            hits += 1
            if is_negated:
                pos += 0.8 * intensity * flip_scale
            else:
                neg += 1.0 * intensity

        i += 1

    score = pos - neg
    return {'positive': pos, 'negative': neg, 'score': score, 'hits': hits}


def assign_sentiment_label(score, hits, thr=0.25):
    if hits == 0:
        return 'Netral', 0.0

    norm = score / max(1.0, float(hits))
    if norm > thr:
        return 'Positif', abs(norm)
    elif norm < -thr:
        return 'Negatif', abs(norm)
    else:
        return 'Netral', abs(norm)


def label_sentiment_lexicon(text, strong_only=False, strong_thr=0.45, thr=0.25):
    result = calculate_sentiment_score(
        text,
        POSITIVE_LEXICON,
        NEGATIVE_LEXICON,
        NEGATION_WORDS,
        INTENSIFIER_WORDS,
        neg_window=3,
        intens_window=2,
        soft_negators=SOFT_NEGATORS,
        soft_factor=0.6
    )

    label, conf = assign_sentiment_label(result['score'], result['hits'], thr=thr)

    if strong_only and conf < strong_thr:
        return None

    return {
        'sentiment': label,
        'confidence': conf,
        'score': result['score'],
        'hits': result['hits'],
        'positive_score': result['positive'],
        'negative_score': result['negative'],
    }

In [ ]:
import string

URL_RE = re.compile(r'http\S+|www\S+|https\S+')
MENTION_RE = re.compile(r'@\w+')
HASHTAG_RE = re.compile(r'#(\w+)')
DIGIT_RE = re.compile(r'\d+')
REPEAT_RE = re.compile(r'(.)\1{2,}')
WS_RE = re.compile(r'\s+')

EMOJI_RE = re.compile("["
    u"\U0001F600-\U0001F64F"
    u"\U0001F300-\U0001F5FF"
    u"\U0001F680-\U0001F6FF"
    u"\U0001F1E0-\U0001F1FF"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
"]+", flags=re.UNICODE)

# pisahkan yang frasa dan yang token
PHRASE_SLANG = {k: v for k, v in SLANG_DICT.items() if ' ' in k}
TOKEN_SLANG  = {k: v for k, v in SLANG_DICT.items() if ' ' not in k}

PHRASE_PATTERNS = [
    (re.compile(r'\b' + re.escape(k) + r'\b'), v)
    for k, v in PHRASE_SLANG.items()
]

# whitelist token penting supaya tidak terhapus stopwords
KEEP_TOKENS = set()
ALL_TERMS = (
    set(POSITIVE_LEXICON)
    | set(NEGATIVE_LEXICON)
    | set(NEGATION_WORDS)
    | set(SOFT_NEGATORS)
    | set(INTENSIFIER_WORDS.keys())
)

for term in ALL_TERMS:
    for w in str(term).split():
        KEEP_TOKENS.add(w)

# punctuation jadi spasi
PUNCT_TABLE = str.maketrans({c: " " for c in string.punctuation})

def preprocess_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""

    text = text.lower()

    # buang url/mention, hashtag tetap jadi kata
    text = URL_RE.sub(" ", text)
    text = MENTION_RE.sub(" ", text)
    text = HASHTAG_RE.sub(r"\1", text)

    # handle slang frasa
    for pat, repl in PHRASE_PATTERNS:
        text = pat.sub(repl, text)

    # handle emoji dan angka
    text = EMOJI_RE.sub(" ", text)
    text = DIGIT_RE.sub(" ", text)

    # handle punctuation, huruf berulang, spasi
    text = text.translate(PUNCT_TABLE)
    text = REPEAT_RE.sub(r"\1", text)
    text = WS_RE.sub(" ", text).strip()
    if not text:
        return ""

    tokens = text.split()

    # handle slang token (1 kata)
    tokens = [TOKEN_SLANG.get(t, t) for t in tokens]

    # handle kalau hasil mapping jadi multi-kata, pecah lagi
    tokens = " ".join(tokens).split()

    # handle stopword removal + whitelist token penting
    tokens = [t for t in tokens if (t in KEEP_TOKENS) or (t not in STOPWORDS_ID)]

    return " ".join(tokens)

In [ ]:
from tqdm import tqdm

tqdm.pandas(desc="Preprocessing")

df["review_clean"] = df["review_text"].progress_apply(preprocess_text)

# buang yang kosong
df = df[df["review_clean"].fillna("").str.strip().ne("")].reset_index(drop=True)

print("Jumlah data setelah preprocess:", len(df))
df[["review_text", "review_clean"]].head()

Preprocessing: 100%|██████████| 71739/71739 [00:01<00:00, 38283.48it/s]


Jumlah data setelah preprocess: 71717


,review_text,review_clean
0,"aplikasi buruk, merugikan, abal abal, duit say...",buruk merugikan abal abal duit hilang topup e ...
1,"Bank besar,tp aplikasinya payah.KTP baret diki...",bank besar aplikasinya buruk ktp baret dikit t...
2,"setiap login disuruh masukan KTP lagi, jadi se...",login disuruh masukan ktp hari bisa x transaks...
3,"Kenapa akhir2 ini brimo di hp saya, selalu set...",akhir brimo hp login berubah putih layarnya ba...
4,Saya kurangi 1 bintang sebab untuk login finge...,kurangi login finger handphone realme tidak mu...


In [ ]:
tqdm.pandas(desc="Strong Pseudo-Labeling")

def get_strong_binary_label(text):
    res = label_sentiment_lexicon(text, strong_only=True, strong_thr=0.60, thr=0.30)
    if res is None:
        return None
    if res["sentiment"] not in ("Positif", "Negatif"):
        return None
    if res["hits"] < 2:
        return None
    return res

df_lex = df.copy()
df_lex["lex"] = df_lex["review_clean"].progress_apply(get_strong_binary_label)
df_lex = df_lex[df_lex["lex"].notna()].reset_index(drop=True)

df_lex["sentiment"]  = df_lex["lex"].apply(lambda d: d["sentiment"])
df_lex["confidence"] = df_lex["lex"].apply(lambda d: d["confidence"])
df_lex["hits"]       = df_lex["lex"].apply(lambda d: d["hits"])

print(df_lex["sentiment"].value_counts())
print("Total kandidat training:", len(df_lex))

Strong Pseudo-Labeling: 100%|██████████| 71717/71717 [00:02<00:00, 29059.39it/s]

sentiment
Negatif    15957
Positif    12853
Name: count, dtype: int64
Total kandidat training: 28810


### Training Data

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier

SKEMA_CONFIG = {
    "Skema 1": {
        "model": "LinearSVC",
        "model_class": LinearSVC(C=1.2, class_weight="balanced", max_iter=20000, dual=True),
        "word_ngram": (1, 2),
        "word_max_features": 40000,
        "word_min_df": 2,
        "word_max_df": 0.90,
        "char_ngram": (3, 5),
        "char_max_features": 60000,
        "char_min_df": 2,
        "char_max_df": 0.95,
        "split": 0.20,
    },
    "Skema 2": { 
        "model": "SGDClassifier(hinge)",
        "model_class": SGDClassifier(
            loss="hinge",
            alpha=1e-5,
            penalty="l2",
            max_iter=2000,
            tol=1e-3,
            early_stopping=True,
            n_iter_no_change=5,
            class_weight="balanced",
            random_state=42,
        ),
        "word_ngram": (1, 2),
        "word_max_features": 50000,
        "word_min_df": 2,
        "word_max_df": 0.90,
        "char_ngram": (3, 5),
        "char_max_features": 60000,
        "char_min_df": 2,
        "char_max_df": 0.95,
        "split": 0.20,
    },
    "Skema 3": {
        "model": "LogisticRegression(saga)",
        "model_class": LogisticRegression(
            C=2.0, max_iter=8000, class_weight="balanced",
            solver="saga", random_state=42, n_jobs=-1
        ),
        "word_ngram": (1, 2),
        "word_max_features": 50000,
        "word_min_df": 2,
        "word_max_df": 0.90,
        "char_ngram": (3, 5),
        "char_max_features": 70000,
        "char_min_df": 2,
        "char_max_df": 0.95,
        "split": 0.20,
    },
}

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# pakai df_lex karena hasil strong pseudo-label
X = df_lex["review_clean"].astype(str).values
y = df_lex["sentiment"].astype(str).values
w = df_lex["confidence"].astype(float).values

le = LabelEncoder()
y_enc = le.fit_transform(y)

def make_union(cfg):
    word_vec = TfidfVectorizer(
        analyzer="word",
        ngram_range=cfg["word_ngram"],
        min_df=cfg["word_min_df"],
        max_df=cfg["word_max_df"],
        max_features=cfg["word_max_features"],
        sublinear_tf=True,
        lowercase=False,
    )
    char_vec = TfidfVectorizer(
        analyzer="char_wb",
        ngram_range=cfg["char_ngram"],
        min_df=cfg["char_min_df"],
        max_df=cfg["char_max_df"],
        max_features=cfg["char_max_features"],
        sublinear_tf=True,
        lowercase=False,
    )
    return FeatureUnion([("word", word_vec), ("char", char_vec)])

X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
    X, y_enc, w, test_size=0.20, random_state=42, stratify=y_enc
)

results = {}

print(f"{'Skema':<8} {'Model':<22} {'TrainAcc':>9} {'TestAcc':>9} {'F1macro':>9} {'PrecM':>9} {'RecM':>9}")
print("-"*80)

for name, cfg in SKEMA_CONFIG.items():
    pipe = Pipeline([
        ("vec", make_union(cfg)),
        ("clf", cfg["model_class"])
    ])

    pipe.fit(X_train, y_train, clf__sample_weight=w_train)

    pred_train = pipe.predict(X_train)
    pred_test  = pipe.predict(X_test)

    train_acc = accuracy_score(y_train, pred_train)
    test_acc  = accuracy_score(y_test, pred_test)
    f1m   = f1_score(y_test, pred_test, average="macro")
    precm = precision_score(y_test, pred_test, average="macro", zero_division=0)
    recm  = recall_score(y_test, pred_test, average="macro", zero_division=0)

    results[name] = {
        "pipe": pipe,
        "train_acc": train_acc,
        "test_acc": test_acc,
        "f1_macro": f1m
    }

    print(f"{name:<8} {cfg['model']:<22} {train_acc:9.4f} {test_acc:9.4f} {f1m:9.4f} {precm:9.4f} {recm:9.4f}")

Skema    Model                   TrainAcc   TestAcc   F1macro     PrecM      RecM
--------------------------------------------------------------------------------
Skema 1  LinearSVC                 1.0000    0.9875    0.9873    0.9879    0.9869
Skema 2  SGDClassifier(hinge)      0.9987    0.9859    0.9858    0.9861    0.9855
Skema 3  LogisticRegression(saga)    0.9946    0.9861    0.9859    0.9867    0.9853


### inferensi

In [ ]:
# ambil skema terbaik dari test_acc dan f1_macro
best_name = max(results, key=lambda k: (results[k]["test_acc"], results[k]["f1_macro"]))
best_pipe = results[best_name]["pipe"]

print("BEST SKEMA:", best_name, "| TestAcc:", round(results[best_name]["test_acc"], 4))

BEST SKEMA: Skema 1 | TestAcc: 0.9875


In [ ]:
import numpy as np

def predict_sentiment(text, pipeline, label_encoder, do_preprocess=True):
    cleaned = preprocess_text(text) if do_preprocess else text
    pred_code = pipeline.predict([cleaned])[0]
    pred_label = label_encoder.inverse_transform([pred_code])[0]
    return {
        "input_text": text,
        "cleaned_text": cleaned,
        "sentiment": pred_label,
        "prediction_code": int(pred_code)
    }

np.random.seed(42)
idxs = np.random.choice(len(X_test), size=10, replace=False)

rows = []
for i in idxs:
    text_clean = X_test[i]
    actual = le.inverse_transform([y_test[i]])[0]
    pred = predict_sentiment(text_clean, best_pipe, le, do_preprocess=False)["sentiment"]
    rows.append({"review_clean": text_clean, "actual": actual, "predicted": pred, "correct": actual == pred})

df_infer = pd.DataFrame(rows)
df_infer

,review_clean,actual,predicted,correct
0,gimn ktnya link d krm k kontak sms nga masuk h...,Negatif,Negatif,True
1,sangat membantu mo transaksi apapun lancar tan...,Positif,Positif,True
2,pendaftaran tolong bisa permudah coba daftar u...,Positif,Positif,True
3,selama mudah aman semoga aman,Positif,Positif,True
4,menyebalkan tadi siang coba video rekam detik ...,Negatif,Negatif,True
5,aman nyaman yang pasti cocok yang punya mobili...,Positif,Positif,True
6,bertransaksi brimo tidak bisa saldo apakah sal...,Negatif,Negatif,True
7,sejauh aplikasinya sangat bagus transaksi muda...,Positif,Positif,True
8,brimo sekarang sulit banget tulisan keluar dul...,Negatif,Negatif,True
9,update tampilan lebih fresh error tulisan gene...,Negatif,Negatif,True
